In [63]:
"""Cut Labels

Demonstrates how to use cut labels to simplify timestamps and streamline usage of LUSID when used across multiple timezones.

Attributes
----------
instruments
transactions
cut labels
holdings
"""

'Cut Labels\n\nDemonstrates how to use cut labels to simplify timestamps and streamline usage of LUSID when used across multiple timezones.\n\nAttributes\n----------\ninstruments\ntransactions\ncut labels\nholdings\n'

# Cut Labels

Cut Labels in LUSID are used to simplify timestamps and streamline the usage of LUSID in a global scenario across multiple timezones.

Instead of providing a date, time, and time zone in every instance, LUSID can be provided with a date and a Cut Label - a user-defined tag describing a timezone and a specific local time in that time zone.

Cut Labels can be thought of as "Custom User Time" Labels, representing daily cut-off times.

For example:

- UTC offset time: 2018-08-31T17:00:00.000000+08:00
- Cut label, assuming SingaporeClose has been configured: 2018-08-31NSingaporeClose

Find out more about Cut Labels in LUSID here:
[LUSID Knowledge Base: Cut Labels](https://support.lusid.com/cut-labels)

This example will cover how to set up and customise Cut Labels, and how to use these in LUSID. 

This example will outline how to:


1. Set Up Cut Labels
	- 1.1. Create Cut Labels
	- 1.2 Cut Label Details
	- 1.3. Delete a Cut Label
2. Create a Transaction Portfolio
3. Add your Instrument Universe
4. Set your Initial Holdings
5. Get Output Transactions
6. Add your Transactions Throughout the Day
7. Retrieve your Transactions
8. Retrieve your Holdings
9. Delete Cut Labels and Portfolios

*First, run the cell below to import libraries and create the LUSID client to initialise our environment*

In [64]:
import os
from datetime import datetime
# Import Libraries
from datetime import timedelta, date

# Import LUSID
import lusid
import lusid.models as models
import pandas as pd
import pytz
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import printer as prettyprint

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.6.8794.0


### Initial Setup
The cell below contains a number of helper functions used throughout this notebook - these are not necessary for any of the methods demonstrated, but simplify the readability and usability of this module

In [65]:
# The following function creates a random alphanumeric code of 4 characters that can be appended to Ids
# and Names to ensure they remain unique throughout multiple runs of this example
def get_guid():
    return datetime.utcnow().strftime("%f")[:4]

Later in this example we will create multiple cut labels in order to demonstrate their use. Note that for these cut labels we have used a range of different time zones. A full list of all accepted timezone names can be found here (under the column *"Zone ID"* or *"Aliases"*): [IANA Time Zone Information](https://nodatime.org/TimeZones)

> Note that we have purposefully set the time for *LondonOpen* as 7am, which we will instead want to be 9am like *SingaporeOpen* and *NewYorkOpen*, and that we have created a dummy cut label *"DeleteMe"* - we will come back to these later.

*Run the cell below to load a csv of cut label data*

In [66]:
# Load CSV file to create more cut labels from
data = pd.read_csv("data/dummy_cut_labels.csv")
data

,display_name,hours,minutes,time_zone
0,LondonOpen,7,0,GB
1,LondonMidday,12,0,GB
2,SingaporeOpen,9,0,Singapore
3,SingaporeMidday,12,0,Singapore
4,SingaporeClose,17,0,Singapore
5,NewYorkOpen,9,0,America/New_York
6,NewYorkMidday,12,0,America/New_York
7,NewYorkClose,17,0,America/New_York
8,DeleteMe,0,0,Europe/Berlin


## 1. Set Up Cut Labels

### 1.1. Create Cut Labels

We begin by creating a single cut label, named *"LondonClose"*, by defining the parameters of the cut label and inserting this into LUSID. We are also going to create a dictionary to store our codes, so that we can call them using *code["display_name"]*.

*Run the cell below to create a cut label and print its information*

In [67]:
# Create the time for the cut label
londonTime1 = models.CutLocalTime(hours=17, minutes=0)

# Create a dictionary to keep track of our cut label codes
code = {}
cut_labels = {}

def upsert_cut_label_definition(
    code: str, 
    description: str, 
    display_name: str, 
    cut_local_time: lusid.models.cut_local_time.CutLocalTime, 
    time_zone:str
):
    # Upsert the request to LUSID to create the cut label
    try:
        # Define the parameters of the cut label in a request
        request = models.CutLabelDefinition(
            code=code, 
            description=description, 
            display_name=display_name,
            cut_local_time=cut_local_time,
            time_zone=time_zone)
        api_factory.build(lusid.api.CutLabelDefinitionsApi).create_cut_label_definition(
            create_cut_label_definition_request=request)

    except lusid.exceptions.ApiException as e:
        request = models.UpdateCutLabelDefinitionRequest(
            display_name=display_name,
            description=description,
            cut_local_time=cut_local_time,
            time_zone=time_zone
        )
        api_factory.build(lusid.api.CutLabelDefinitionsApi).update_cut_label_definition(
            code=code,
            update_cut_label_definition_request=request
        )

    cut_labels[code] = code

    display(f"added {code}")

    return code, request.display_name


guid = get_guid()
        
cut_label_code, disp_name = upsert_cut_label_definition(
    code=f"LondonClose-{guid}",
    description="London Closing Time, 5pm in UK",
    display_name="LondonClose",
    cut_local_time=londonTime1,
    time_zone="GB"
)

# Add the codes of our cut labels to our dictionary
code[disp_name] = cut_label_code 

'added LondonClose-7316'

Now we will create multiple more cut labels from the csv data by the same method, and then list the details of all of our existing cut labels.

These cut labels will represent the Open, Midday, and Closing times for 3 offices - one in London, one in Singapore, and one in New York, and we will later use these to make transactions. 

See below for a timeline of the working days of each office, compared to the London time along the bottom axis:

![Init](img/cut-labels-timezones.png)

Making transactions across all of these timezones can get confusing, particularly when considering the holdings at different times and when each transaction occured in each time zone. Using our cut labels we will later demonstrate the bi-temporal features of LUSID to make this process simpler.  

Read more about bi-temporal data here: [LUSID Knowledge base: Bi-Temporal Data](https://support.lusid.com/what-is-bi-temporal-data)

*Run the cell below to create cut labels and print their information*

In [68]:
# Create cut labels from the loaded CSV data
for index, row in data.iterrows():

    cut_label_code, disp_name = upsert_cut_label_definition(
        code=row["display_name"]+"-"+guid,
        description="",
        display_name=row["display_name"],
        cut_local_time=models.CutLocalTime(
            hours=row["hours"], 
            minutes=row["minutes"]),
        time_zone=row["time_zone"]
    )
    # Add the codes of our cut labels to our dictionary
    code[disp_name] = cut_label_code 

# List the details of all existing cut labels
response = api_factory.build(lusid.api.CutLabelDefinitionsApi).list_cut_label_definitions()
lusid_response_to_data_frame(response.values)

'added LondonOpen-7316'

'added LondonMidday-7316'

'added SingaporeOpen-7316'

'added SingaporeMidday-7316'

'added SingaporeClose-7316'

'added NewYorkOpen-7316'

'added NewYorkMidday-7316'

'added NewYorkClose-7316'

'added DeleteMe-7316'

,code,display_name,description,cut_local_time.hours,cut_local_time.minutes,time_zone
0,DeleteMe-7316,DeleteMe,,0,0,Europe/Berlin
1,LondonClose-7316,LondonClose,"London Closing Time, 5pm in UK",17,0,GB
2,LondonMidday-7316,LondonMidday,,12,0,GB
3,LondonOpen-7316,LondonOpen,,7,0,GB
4,NewYorkClose-7316,NewYorkClose,,17,0,America/New_York
5,NewYorkMidday-7316,NewYorkMidday,,12,0,America/New_York
6,NewYorkOpen-7316,NewYorkOpen,,9,0,America/New_York
7,SingaporeClose-7316,SingaporeClose,,17,0,Singapore
8,SingaporeMidday-7316,SingaporeMidday,,12,0,Singapore
9,SingaporeOpen-7316,SingaporeOpen,,9,0,Singapore


### 1.2 Cut Label Details

#### 1.2.1 Get Cut Label Details
We are now going to look closer at our LondonOpen cut label.

*Run the cell below to get details on the cut label "LondonOpen"*

In [69]:
# First list the original details for the cut label we are going to change
details = api_factory.build(lusid.api.CutLabelDefinitionsApi).get_cut_label_definition(
    code=code["LondonOpen"])
prettyprint.get_cut_label(details)

Cut Label Details:
Display Name: LondonOpen
Code: LondonOpen-7316
Local Time: 7:00
Timezone: GB
Description: 



#### 1.2.2. Update a Cut Label
Looking closely, we can see that the Local Time for this cut label is wrong - it should be 9am. <br/>
We can now modify this, and we will add a description to this cut label

*Run the cell below to modify the details of the cut label*

In [70]:
# Modify London Open cut label time to 9:00am
londonTime2 = models.CutLocalTime(hours=9, minutes=0)

request = models.CutLabelDefinition(
    description="London Opening Time, 9am in UK",
    display_name="LondonOpen",
    cut_local_time=londonTime2,
    time_zone="GB")

response = api_factory.build(lusid.api.CutLabelDefinitionsApi).update_cut_label_definition(
    code=code[f"LondonOpen"],
    update_cut_label_definition_request=request)

# Print the updated London Close cut label
prettyprint.update_cut_label(response)

Updated Cut Label:
Display Name: LondonOpen
Code: LondonOpen-7316
Local Time: 9:00
Timezone: GB
Description: London Opening Time, 9am in UK



### 1.3. Delete a Cut Label
When we created our cut labels, we also created one for an obsolete time, helpfully named *"DeleteMe"*.

We will now delete this cut label and list our cut labels again - DeleteMe has been deleted!

*Run the cell below to delete the cut label*

In [71]:
# Delete cut label "Delete Me"
try:
    api_factory.build(lusid.api.CutLabelDefinitionsApi).delete_cut_label_definition(code=code["DeleteMe"])
    code.pop("DeleteMe")
except lusid.ApiException as ex:
    if int(ex.status) == 404:
        print("404 error when deleting cut label \"DeleteMe\"")
    else:
        raise ex

# List the details of all existing cut labels
response = api_factory.build(lusid.api.CutLabelDefinitionsApi).list_cut_label_definitions()
lusid_response_to_data_frame(response.values)

,code,display_name,description,cut_local_time.hours,cut_local_time.minutes,time_zone
0,LondonClose-7316,LondonClose,"London Closing Time, 5pm in UK",17,0,GB
1,LondonMidday-7316,LondonMidday,,12,0,GB
2,LondonOpen-7316,LondonOpen,"London Opening Time, 9am in UK",9,0,GB
3,NewYorkClose-7316,NewYorkClose,,17,0,America/New_York
4,NewYorkMidday-7316,NewYorkMidday,,12,0,America/New_York
5,NewYorkOpen-7316,NewYorkOpen,,9,0,America/New_York
6,SingaporeClose-7316,SingaporeClose,,17,0,Singapore
7,SingaporeMidday-7316,SingaporeMidday,,12,0,Singapore
8,SingaporeOpen-7316,SingaporeOpen,,9,0,Singapore


## 2. Create a Transaction Portfolio
We are now going to look at some example use cases for our cut labels. To do this we are going to need to create a portfolio to be used in all three of our offices, and in order to create a portfolio you will need to define a scope. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

Once you have a scope, you can create a portfolio inside this scope. Every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

> Note that currently the create portfolio API call does not support cut labels - make sure to check the api docs for *"date-time"* or *"dateorcutlabel"* for each parameter type

*Run the cell below to create your scope and portfolio*

In [72]:
# Create our portfolio
import json
# Create a scope for our portfolio
scope = 'portfolio_demo'

# Set the code of your portfolio
portfolio_code = f"Global-Strategies-{guid}"

# Set the creation date of your portfolio
effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)

try:
    # Next we create a request for the portfolio
    request = models.CreateTransactionPortfolioRequest(
        code=portfolio_code,
        display_name="Global Strategies Fund",
        base_currency="GBP",
        created=effective_date,
        description=None,
        corporate_action_source_id=None,
        accounting_method=None,
        sub_holding_keys=None,
        properties=None)
    # And finally we can upsert the portfolio creation request to LUSID
    result = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=request
    )
    
    # prettyprint the response
    prettyprint.portfolio_response(result)
except lusid.exceptions.ApiException as e:
    body = json.loads(e.body)
    print(body["name"])
        

Portfolio Created
Scope: portfolio_demo
Code: Global-Strategies-7316
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2022-03-11 16:13:13.792878+00:00



We will now also define a function to format our cut label onto a date in a format readable to LUSID

> Date-times are usually in the form "2018-08-31T07:36:53.523415+01:00" 
>
>Cut labels are written in the form "2018-08-31NSingaporeClose"

In [73]:
# Defines a function to write the date and cut label in the format "YYYY-MM-DDN{cut_label_code}""
def cut_label_formatter(date, cut_label_code):
    return str(date) + "N" + cut_label_code

## 3. Add your Instrument Universe
We now want to make trades and take on holdings using our cut labels, but before we can do this we need to populate our instrument universe. In this case we are importing our instrument universe from a CSV file. 

Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [74]:
instrument_universe = pd.read_csv('data/instruments.csv')
instrument_universe.head()

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [75]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in instrument_universe.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = [
            ('isin', 'Isin'), 
            ('figi', 'Figi'), 
            ('ticker', 'Ticker'),
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument defintions
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(request_body=batch_upsert_request)

# Pretty print the response
prettyprint.instrument_response(instrument_response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,Glencore_LondonStockEx_GLEN,imd_34534555,LUID_0000MIPD
1,BP_LondonStockEx_BP,imd_43535553,LUID_00010BYI
2,BurfordCapital_LondonStockEx_BUR,imd_43534356,LUID_0000MIOY
3,MicroFocus_LondonStockEx_MCRO,imd_34567338,LUID_0000MIP9
4,UKGiltTreasury_4.5_2034,imd_34534534,LUID_0000MIPA
5,Sage_LondonStockEx_SGE,imd_23536673,LUID_0000MIP5
6,WPP_LondonStockEx_WPP,imd_34536734,LUID_0000MIPF
7,TESCO_LondonStockEx_TSCO,internal_id_5,LUID_0000NA4Q
8,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,LUID_0000MIPB
9,Amazon_Nasdaq_AMZN,imd_34634534,LUID_0000MIOX


## 4. Set your Initial Holdings
Now that you have your instrument universe populated you can load your initial holdings into your portfolio. In this case you will import your holdings from a CSV file. 

*Run the cell below to import your take on balances*

In [76]:
holdings = pd.read_csv('data/demo-holdings.csv')
holdings.head()

,portfolio_code,instrument_name,quantity,price,currency,figi
0,Global-Strategies,GBP_Cash,8336000,1.00,GBP,NaN
1,Global-Strategies,Glencore_LondonStockEx_GLEN,905141,2.76,GBP,BBG001MM1KV4
2,Global-Strategies,Kingfisher_LondonStockEx_KGF,1362038,2.28,GBP,BBG000BKH1W6
3,Global-Strategies,UKGiltTreasury_2.0_2025,405589,106.64,GBP,BBG0088JSC32
4,Global-Strategies,UKGiltTreasury_3.5_2045,266169,134.43,GBP,BBG006N6HZM7


Now that you have imported your holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. 

Read more about how holdings are generated in LUSID here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-are-holdings-generate-by-lusid).

First we will need to set an effective date from which our holdings are recorded, which we will set as five days ago from the start of the day in our London office, using our *"LondonOpen"* cut label.

> Note that the "Effective From" time outputted is in UTC, so if the date used with the cut label coincides with daylight savings this may appear to be an hour out from our cut label, but this is only because our cut label uses the local time, and this is using UTC

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to set our effective date and upsert your holdings into LUSID*

In [77]:
# Make the holdings effective date
holdings_effective_date = date.today() - timedelta(days=5)
holdings_cut_label_code = code["LondonOpen"]

holdings_effective_cut_label = cut_label_formatter(holdings_effective_date, holdings_cut_label_code)

print("Effective date: " + holdings_effective_cut_label + "\n")

# Iterate the portfolios in the holdings CSV, note in this case you only have one
for portfolio in holdings['portfolio_code'].unique():
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the holdings in each portfolio
    for index, holding in holdings.loc[holdings['portfolio_code'] == portfolio].iterrows():
        
        # Set your instrument identifiers based on whether or not instrument is cash
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/Figi'
            identifer = holding['figi']
            
        # Create your holding adjustment and append it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
    
    # Call LUSID to set your initial holdings
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=scope,
        code=portfolio_code,
        effective_at=holdings_effective_cut_label,
        adjust_holding_request=holding_adjustments)

    # Pretty print our response from LUSID
    prettyprint.set_holdings_response(response, scope, portfolio)

Effective date: 2022-03-06NLondonOpen-7316

Holdings Successfully Set for Portfolio
Scope: portfolio_demo
Code: Global-Strategies
Holdings Effective From: 2018-01-01 00:00:00+00:00
Holdings Created On: 2022-03-11 16:13:17.536169+00:00



## 5. Get Output Transactions
It is possible to see how LUSID has adjusted the holdings of our portfolio to meet our initial holdings by generating the output transactions from the portfolio. You can read more about output transactions here [LUSID Knowledge Base: Output Transactions](https://support.lusid.com/what-is-an-output-transaction-in-lusid).

In order to build these transactions we need to set a start date and an end date - again we will use our London cut labels (LondonOpen and LondonClose) for these, as the holdings were set in the London office, and we will set our start and end dates to go with this cut label as 4 days either side of the effective date of our holdings.

For further usage of the build transactions API call refer to the [LUSID API Docs: Build Output Transactions](https://docs.lusid.com/#operation/BuildTransactions).

*Run the cell below to generate your output transactions*

In [78]:
# Set start date cut label
start_date_label = cut_label_formatter(
        holdings_effective_date-timedelta(days=4),
        code["LondonOpen"])
# Set end date cut label
end_date_label = end_date=cut_label_formatter(
        holdings_effective_date+timedelta(days=4),
        code["LondonClose"])

print("start date: " + start_date_label)
print("end date: " + end_date_label + "\n")

# Set our query parameters to build your transactions
query_params = models.TransactionQueryParameters(
    start_date=start_date_label,
    end_date=end_date_label,
    query_mode='TradeDate',
    show_cancelled_transactions=None)

# Call LUSID to build your output transactions
response = api_factory.build(lusid.api.TransactionPortfoliosApi).build_transactions(
    scope=scope,
    code=portfolio_code,
    property_keys=['Instrument/default/Name'],
    transaction_query_parameters=query_params
)

# Pretty print your output transactions
prettyprint.output_transactions(response, scope, portfolio_code, ['Instrument/default/Name'])

start date: 2022-03-02NLondonOpen-7316
end date: 2022-03-10NLondonClose-7316

Output Transactions for Portfolio
Scope: portfolio_demo
Code: Global-Strategies-7316



,Transaction ID,Transaction Type,Instrument/default/Name,Units,Price,Currency,Transaction Date,Settlement Date,Realised Gain Loss
0,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,GBP,8336000.00,1.00,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
1,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,Glencore_LondonStockEx_GLEN,905141.00,2.76,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
2,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,Kingfisher_LondonStockEx_KGF,1362038.00,2.28,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
3,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,UKGiltTreasury_2.0_2025,405589.00,106.64,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
4,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,UKGiltTreasury_3.5_2045,266169.00,134.43,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
5,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,UKGiltTreasury_3.75_2021,661713.00,108.13,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
6,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,UKGiltTreasury_4.5_2034,77481.00,140.57,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
7,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,USTreasury_2.00_2021,1440244.00,97.90,USD,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
8,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,BurfordCapital_LondonStockEx_BUR,853486.00,14.06,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN
9,2022-03-06T09:00:00.0000000+00:00,AdjustmentIncrease,EKFDiagnostics_LondonStockEx_EKF,925925.00,0.27,GBP,2022-03-06 09:00:00+00:00,2022-03-06 09:00:00+00:00,NaN


## 6. Add your Transactions Throughout the Day
Now that you have set your initial holdings you are ready to take on some transactions. You have made several trades over the last day in each office that you would like to add to LUSID. You will import these from a CSV file and split them up into the different regions. Note that this can also be done by importing 3 separate CSV files, or any other data store.

*Run the cell below to import your transactions*

In [79]:
daily_transactions = pd.read_csv('data/demo-transactions-2.csv')

daily_transactions_LDN =  daily_transactions.iloc[0:2,:]
daily_transactions_NY = daily_transactions.iloc[2:4,:]
daily_transactions_SG = daily_transactions.iloc[4:6,:]

daily_transactions.head()

,portfolio_code,transaction_id,instrument_name,transaction_description,transaction_type,transaction_units,transaction_price,transaction_currency,transaction_strategy,transaction_cost,figi
0,Global-Strategies,tid_329432525234324,Kingfisher_LondonStockEx_KGF,Equity Sale,Sell,325000,2.35,GBP,quantitativeSignal,762125.00,BBG000BKH1W6
1,Global-Strategies,tid_325452342424500,UKGiltTreasury_4.5_2034,Equity Purchase,Buy,10501,140.57,GBP,incomeRequirements,1476146.57,BBG0000D14P3
2,Global-Strategies,tid_234295929052090,UKGiltTreasury_3.75_2021,Equity Purchase,Buy,24000,109.13,GBP,incomeRequirements,2619024.00,BBG001KKJLR4
3,Global-Strategies,tid_234942982496001,USTreasury_2.00_2021,Equity Sale,Sell,57000,97.80,USD,internationalExposure,5574600.00,BBG00FN3B5K8
4,Global-Strategies,tid_121309590059995,BP_LondonStockEx_BP,Equity Purchase,Buy,50000,5.29,GBP,fundamentalAnalysis,264350.00,BBG000C05BD1


Now that we have imported our transactions we can upsert them into LUSID. 

We will define a function to do this, and then upsert the transactions from each timezone using the respective cut labels to set our transaction and settlement dates.

Read more about transactions here [LUSID Knowledge Base: Transactions](https://support.lusid.com/what-is-a-transaction). 

> Again note that the "Effective From" time outputted is in UTC, so if the date used with the cut label coincides with daylight savings this may appear to be an hour out from our cut label and the times shown in our diagram, but this is only because our cut label uses the local time, and this is using UTC

For further usage of the upsert transactions API call refer to the [LUSID API Docs: Upsert Transactions](https://docs.lusid.com/#operation/UpsertTransactions).

*Run the cell below to define our function to upsert transactions*

In [80]:
# Set your transaction and settlement dates
transaction_date = date.today()-timedelta(days=1)
settlement_date = transaction_date+timedelta(days=2)

def add_transaction(cut_label_display_name, timezone_transactions):
    # Turn them into cut labels
    transaction_cut_label = cut_label_formatter(transaction_date, code[cut_label_display_name])
    settlement_cut_label = cut_label_formatter(settlement_date, code[cut_label_display_name])

    print("Transaction date: " + transaction_cut_label)
    print("Settlement date: " + settlement_cut_label + "\n")

    # Iterate over the portfolios in our transactions file, in this case only one
    for portfolio in timezone_transactions['portfolio_code'].unique():

        # Initialise a list to hold your transaction requests
        transactions = []

        # Iterate over the transactions in the portfolio
        for index, transaction in timezone_transactions.loc[timezone_transactions['portfolio_code'] == portfolio].iterrows():

            # Append your request to the list
            transactions.append(models.TransactionRequest(
                transaction_id=transaction['transaction_id'],
                type=transaction['transaction_type'],
                instrument_identifiers={
                    'Instrument/default/Figi': transaction['figi']
                },
                transaction_date=transaction_cut_label,
                settlement_date=settlement_cut_label,
                units=transaction['transaction_units'],
                transaction_price=models.TransactionPrice(
                      price=transaction['transaction_price'],
                      type='Price'),
                total_consideration=models.CurrencyAndAmount(
                  amount=transaction['transaction_cost'],
                  currency=transaction['transaction_currency']),
                 source='Default',
                 transaction_currency=transaction['transaction_currency']))

        # Call LUSID to upsert your transactions for this portfolio
        response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transaction_request=transactions)

        # Print the response from LUSID using pretty formatting 
        prettyprint.transactions_response(
            response,
            scope,
            portfolio_code)

*Run the cell below to upsert our London transactions into LUSID*

In [81]:
add_transaction("LondonClose", daily_transactions_LDN)

Transaction date: 2022-03-10NLondonClose-7316
Settlement date: 2022-03-12NLondonClose-7316

Transactions Successfully Upserted into Portfolio
Scope: portfolio_demo
Code: Global-Strategies-7316
Transactions Effective From: 2022-03-10 17:00:00+00:00
Transactions Created On: 2022-03-11 16:13:20.438613+00:00



*Run the cell below to upsert our New York transactions into LUSID*

In [82]:
add_transaction("NewYorkClose", daily_transactions_NY)

Transaction date: 2022-03-10NNewYorkClose-7316
Settlement date: 2022-03-12NNewYorkClose-7316

Transactions Successfully Upserted into Portfolio
Scope: portfolio_demo
Code: Global-Strategies-7316
Transactions Effective From: 2022-03-10 22:00:00+00:00
Transactions Created On: 2022-03-11 16:13:21.578316+00:00



*Run the cell below to upsert our Singapore transactions into LUSID*

In [83]:
add_transaction("SingaporeClose", daily_transactions_SG)

Transaction date: 2022-03-10NSingaporeClose-7316
Settlement date: 2022-03-12NSingaporeClose-7316

Transactions Successfully Upserted into Portfolio
Scope: portfolio_demo
Code: Global-Strategies-7316
Transactions Effective From: 2022-03-10 09:00:00+00:00
Transactions Created On: 2022-03-11 16:13:22.575311+00:00



## 7. Retrieve your Transactions
So now we've made our transactions, but because they all occurred at different times in different time zones, it can be difficult to figure out when transactions from one time zone occurred in another. Each of our transactions occur at the closing time of each office in their respective time zones (again noting that the *"Effective From"* above is in UTC).

We can visualise our transactions happening as below, with the transactions for each region shown in green:

![image.png](img/cut-labels-timezones-with-transactions.png)

We can now go back and look at the transactions countained in our portfolio, and we can get these transactions over a given interval of effective time. Therefore we will look at the transactions occuring between the opening and closing times at each of our offices, using our cut labels.

Like when we added our transactions, we will first define a function to do this, and then retrieve our transactions for each of our cut labels.

From the above timeline, we should expect to see just the Singapore transactions over the Singapore working day, the Singapore and London transactions in London, and the London and New York transactions in New York. The good news is - LUSID works this all out for us!

For further usage of the get transactions API call refer to the [LUSID API Docs: Get Transactions](https://docs.lusid.com/#operation/GetTransactions).
                                                                                                   
*Run the cell below to define our function to get your transactions*

In [84]:
def retrieve_transactions(open_cut_label_display_name, close_cut_label_display_name):
    open_formatted = cut_label_formatter(transaction_date, code[open_cut_label_display_name])
    close_formatted = cut_label_formatter(transaction_date, code[close_cut_label_display_name])

    # Call LUSID to get your transactions made between these cut labels
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_transactions(
        scope=scope,
        code=portfolio_code,
        from_transaction_date=open_formatted, 
        to_transaction_date=close_formatted)

    # Pretty print the response
    print("Transactions occuring during London working day: \n")
    return prettyprint.get_transactions_response(
        response,
        scope,
        portfolio_code,
        [])

*Run the cell below to get your transactions throughout the day in London*

In [85]:
retrieve_transactions("LondonOpen", "LondonClose")

Transactions occuring during London working day: 

Transactions Retrieved from Portfolio
Scope:  portfolio_demo
Code:  Global-Strategies-7316 



,Transaction ID,Transaction Type,Transaction/default/SourcePortfolioId,Transaction/default/SourcePortfolioScope,Units,Price,Currency,Transaction Date
0,tid_121309590059995,Buy,Global-Strategies-7316,portfolio_demo,50000.00,5.29,GBP,2022-03-10 09:00:00+00:00
1,tid_124319009950924,Sell,Global-Strategies-7316,portfolio_demo,-28000.00,98.10,USD,2022-03-10 09:00:00+00:00
2,tid_329432525234324,Sell,Global-Strategies-7316,portfolio_demo,325000.00,2.35,GBP,2022-03-10 17:00:00+00:00
3,tid_325452342424500,Buy,Global-Strategies-7316,portfolio_demo,10501.00,140.57,GBP,2022-03-10 17:00:00+00:00


*Run the cell below to get your transactions throughout the day in New York*

In [86]:
retrieve_transactions("NewYorkOpen", "NewYorkClose")

Transactions occuring during London working day: 

Transactions Retrieved from Portfolio
Scope:  portfolio_demo
Code:  Global-Strategies-7316 



,Transaction ID,Transaction Type,Transaction/default/SourcePortfolioId,Transaction/default/SourcePortfolioScope,Units,Price,Currency,Transaction Date
0,tid_329432525234324,Sell,Global-Strategies-7316,portfolio_demo,325000.00,2.35,GBP,2022-03-10 17:00:00+00:00
1,tid_325452342424500,Buy,Global-Strategies-7316,portfolio_demo,10501.00,140.57,GBP,2022-03-10 17:00:00+00:00
2,tid_234295929052090,Buy,Global-Strategies-7316,portfolio_demo,24000.00,109.13,GBP,2022-03-10 22:00:00+00:00
3,tid_234942982496001,Sell,Global-Strategies-7316,portfolio_demo,57000.00,97.80,USD,2022-03-10 22:00:00+00:00


*Run the cell below to get your transactions throughout the day in Singapore*

In [87]:
retrieve_transactions("SingaporeOpen", "SingaporeClose")

Transactions occuring during London working day: 

Transactions Retrieved from Portfolio
Scope:  portfolio_demo
Code:  Global-Strategies-7316 



,Transaction ID,Transaction Type,Transaction/default/SourcePortfolioId,Transaction/default/SourcePortfolioScope,Units,Price,Currency,Transaction Date
0,tid_121309590059995,Buy,Global-Strategies-7316,portfolio_demo,50000.00,5.29,GBP,2022-03-10 09:00:00+00:00
1,tid_124319009950924,Sell,Global-Strategies-7316,portfolio_demo,-28000.00,98.10,USD,2022-03-10 09:00:00+00:00


## 8. Retrieve your Holdings
We can see how these transactions have affected your holdings by trying to retrieve the holdings of your portfolio. 

Again, it can be quite difficult to see how our transactions have changed our initial holdings at different times in each time zone, but again LUSID does the hard work for us - all we need to do is provide the cut label for the time at which we would like to retrieve our holdings. 

We will retrieve the holdings at the closing time for each office using our cut labels by first defining our function, and then passing in our cut labels.

At each closing time, the holdings will be the initial holdings plus the modifications due to each transaction occuring *before* the closing time (the time at which the holdings are retrieved) - the above diagram should help clarify this.

For further usage of the get holdings API call refer to the [LUSID API Docs: Get Holdings](https://docs.lusid.com/#operation/GetHoldings).

*Run the cell below to get our holdings from our portfolio at the end of the day in London*

In [88]:
def retrieve_holdings(cut_label_display_name):
    cut_label_formatted = cut_label_formatter(transaction_date, code[cut_label_display_name])
    
    result = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
        scope=scope,
        code=portfolio_code,
        property_keys=['Instrument/default/Name'],
        effective_at=cut_label_formatted)

    print("Effective at: " + cut_label_formatted + "\n")
    return prettyprint.holdings_response(result, scope, portfolio_code)

At London Close, our holdings will include the modifications from both the Singapore and London transactions.

*Run the cell below to get our holdings from our portfolio at the end of the day in London*

In [89]:
retrieve_holdings("LondonClose")

Effective at: 2022-03-10NLondonClose-7316

Holdings for Portfolio
Scope: portfolio_demo
Code: Global-Strategies-7316



,Instrument/default/Name,Holding/default/SourcePortfolioId,Holding/default/SourcePortfolioScope,Units,Cost,Currency,Unsettled Transaction Id,Settlement Date
0,GBP,Global-Strategies-7316,portfolio_demo,8336000.00,8336000.00,GBP,-,-
1,Glencore_LondonStockEx_GLEN,Global-Strategies-7316,portfolio_demo,905141.00,2499999.44,GBP,-,-
2,Kingfisher_LondonStockEx_KGF,Global-Strategies-7316,portfolio_demo,1037038.00,2360298.49,GBP,-,-
3,UKGiltTreasury_2.0_2025,Global-Strategies-7316,portfolio_demo,405589.00,43250794.19,GBP,-,-
4,UKGiltTreasury_3.5_2045,Global-Strategies-7316,portfolio_demo,266169.00,35781897.18,GBP,-,-
5,UKGiltTreasury_3.75_2021,Global-Strategies-7316,portfolio_demo,661713.00,71548379.84,GBP,-,-
6,UKGiltTreasury_4.5_2034,Global-Strategies-7316,portfolio_demo,87982.00,12367805.70,GBP,-,-
7,USTreasury_2.00_2021,Global-Strategies-7316,portfolio_demo,1468244.00,143746687.60,USD,-,-
8,BurfordCapital_LondonStockEx_BUR,Global-Strategies-7316,portfolio_demo,853486.00,12000013.16,GBP,-,-
9,EKFDiagnostics_LondonStockEx_EKF,Global-Strategies-7316,portfolio_demo,925925.00,249999.75,GBP,-,-


At New York Close, our holdings will include the modifications from all our transactions.

*Run the cell below to get our holdings from our portfolio at the end of the day in New York*

In [90]:
retrieve_holdings("NewYorkClose")

Effective at: 2022-03-10NNewYorkClose-7316

Holdings for Portfolio
Scope: portfolio_demo
Code: Global-Strategies-7316



,Instrument/default/Name,Holding/default/SourcePortfolioId,Holding/default/SourcePortfolioScope,Units,Cost,Currency,Unsettled Transaction Id,Settlement Date
0,GBP,Global-Strategies-7316,portfolio_demo,8336000.00,8336000.00,GBP,-,-
1,Glencore_LondonStockEx_GLEN,Global-Strategies-7316,portfolio_demo,905141.00,2499999.44,GBP,-,-
2,Kingfisher_LondonStockEx_KGF,Global-Strategies-7316,portfolio_demo,1037038.00,2360298.49,GBP,-,-
3,UKGiltTreasury_2.0_2025,Global-Strategies-7316,portfolio_demo,405589.00,43250794.19,GBP,-,-
4,UKGiltTreasury_3.5_2045,Global-Strategies-7316,portfolio_demo,266169.00,35781897.18,GBP,-,-
5,UKGiltTreasury_3.75_2021,Global-Strategies-7316,portfolio_demo,685713.00,74167403.84,GBP,-,-
6,UKGiltTreasury_4.5_2034,Global-Strategies-7316,portfolio_demo,87982.00,12367805.70,GBP,-,-
7,USTreasury_2.00_2021,Global-Strategies-7316,portfolio_demo,1411244.00,138166170.20,USD,-,-
8,BurfordCapital_LondonStockEx_BUR,Global-Strategies-7316,portfolio_demo,853486.00,12000013.16,GBP,-,-
9,EKFDiagnostics_LondonStockEx_EKF,Global-Strategies-7316,portfolio_demo,925925.00,249999.75,GBP,-,-


At Singapore Close, our holdings will include the modifications from just the Singapore transactions.

*Run the cell below to get our holdings from our portfolio at the end of the day in Singapore*

In [91]:
retrieve_holdings("SingaporeClose")

Effective at: 2022-03-10NSingaporeClose-7316

Holdings for Portfolio
Scope: portfolio_demo
Code: Global-Strategies-7316



,Instrument/default/Name,Holding/default/SourcePortfolioId,Holding/default/SourcePortfolioScope,Units,Cost,Currency,Unsettled Transaction Id,Settlement Date
0,GBP,Global-Strategies-7316,portfolio_demo,8336000.00,8336000.00,GBP,-,-
1,Glencore_LondonStockEx_GLEN,Global-Strategies-7316,portfolio_demo,905141.00,2499999.44,GBP,-,-
2,Kingfisher_LondonStockEx_KGF,Global-Strategies-7316,portfolio_demo,1362038.00,3099998.49,GBP,-,-
3,UKGiltTreasury_2.0_2025,Global-Strategies-7316,portfolio_demo,405589.00,43250794.19,GBP,-,-
4,UKGiltTreasury_3.5_2045,Global-Strategies-7316,portfolio_demo,266169.00,35781897.18,GBP,-,-
5,UKGiltTreasury_3.75_2021,Global-Strategies-7316,portfolio_demo,661713.00,71548379.84,GBP,-,-
6,UKGiltTreasury_4.5_2034,Global-Strategies-7316,portfolio_demo,77481.00,10891659.13,GBP,-,-
7,USTreasury_2.00_2021,Global-Strategies-7316,portfolio_demo,1468244.00,143746687.60,USD,-,-
8,BurfordCapital_LondonStockEx_BUR,Global-Strategies-7316,portfolio_demo,853486.00,12000013.16,GBP,-,-
9,EKFDiagnostics_LondonStockEx_EKF,Global-Strategies-7316,portfolio_demo,925925.00,249999.75,GBP,-,-


## 9. Delete Cut Labels and Portfolios
Finally, we will delete all of our existing cut labels and our portfolio, in order to tidy up our LUSID environment from this example and allowing it to be run again in its entirety.

*Run the cells below to clear up your LUSID environment*

In [92]:
# Deletes all cut labels
for cut_label in code.values():
    try:
        api_factory.build(lusid.api.CutLabelDefinitionsApi).delete_cut_label_definition(code=cut_label)
    except lusid.ApiException as ex:
        print(f"error when deleting cut label {cut_label}: {ex}")

# Delete portfolio
try:
    api_factory.build(lusid.api.PortfoliosApi).delete_portfolio(scope=scope, code=portfolio_code)
except lusid.ApiException as ex:
    print(f"error deleting portfolio {scope}/{code}: {ex}")